# Clasificación de cyberbullying con BERT

<img src="../img/bert.png" width="700"/>

__Imagen tomada de Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805.__

## 1.- Conjuntos de datos
- Partición de entrenamiento, validación y prueba.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./cyberbullying_tweets.csv')   

In [3]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [5]:
# Contar elementos por clase
count = df['cyberbullying_type'].value_counts()
count

cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df= train_test_split(df, test_size = 0.30, random_state = 123)

In [7]:
train_df['cyberbullying_type'].value_counts()

cyberbullying_type
gender                 5655
not_cyberbullying      5610
ethnicity              5601
age                    5573
religion               5557
other_cyberbullying    5388
Name: count, dtype: int64

In [8]:
val_df, test_df= train_test_split(test_df, test_size = 0.50, random_state = 123)

In [9]:
val_df['cyberbullying_type'].value_counts()

cyberbullying_type
religion               1250
other_cyberbullying    1219
age                    1198
gender                 1177
ethnicity              1164
not_cyberbullying      1146
Name: count, dtype: int64

In [10]:
test_df['cyberbullying_type'].value_counts()

cyberbullying_type
age                    1221
other_cyberbullying    1216
ethnicity              1196
religion               1191
not_cyberbullying      1189
gender                 1141
Name: count, dtype: int64

## 2.- Preprocesamiento

In [11]:
# Creamos un diccionario que mapea cada etiqueta a un número entero
labels_dict = {
    'not_cyberbullying': 0,
    'gender': 1,
    'religion': 2,
    'other_cyberbullying': 3,
    'age': 4,
    'ethnicity': 5
}

# Usamos la función map() para reemplazar cada etiqueta con su valor entero correspondiente
train_df['cyberbullying_type'] = train_df['cyberbullying_type'].replace(labels_dict)
val_df['cyberbullying_type'] = val_df['cyberbullying_type'].replace(labels_dict)
test_df['cyberbullying_type'] = test_df['cyberbullying_type'].replace(labels_dict)

train_df.head()

,tweet_text,cyberbullying_type
26651,@AntonSirius @erinspice @prpltnkr @ChiefElk oh...,3
4820,"Ladies ""Bedroom Bully"" The Mix Cd By @GappyRan...",0
1847,RT @_bobbidana: Never thought I'd say this but...,0
20653,She is intellectual terrorists and world suffe...,2
19195,You saudias are not friends of Muslim idiots c...,2


- Verifica cadenas vacias.

In [12]:
train_df['tweet_text'].isna().sum()

0

- Elimina puntuación y convierte a minúsculas.
- Se utiliza el método __str.translate()__ para eliminar todos los caracteres de puntuación mediante una tabla de traducción creada con el método __str.maketrans__. La constante string.punctuation contiene todos los caracteres de puntuación ASCII, que se eliminan de los valores en la columna.

In [13]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
train_df['tweet_text'] = train_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
train_df.head()

,tweet_text,cyberbullying_type
26651,antonsirius erinspice prpltnkr chiefelk oh dea...,3
4820,ladies bedroom bully the mix cd by gappyranks ...,0
1847,rt bobbidana never thought id say this but i h...,0
20653,she is intellectual terrorists and world suffe...,2
19195,you saudias are not friends of muslim idiots c...,2


In [15]:
val_df['tweet_text'] = val_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
val_df.head()

,tweet_text,cyberbullying_type
21718,but you idiot tagged and more also im a muslim,2
45048,sexylala thats a dumb nigger never fuck lala h...,5
40415,i never hear hispanics calling each other brow...,5
40212,lennybanx nigger read it again such a dumb ass...,5
35187,she was a mean girl in high school she and mim...,4


In [16]:
test_df['tweet_text'] = test_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
test_df.head()

,tweet_text,cyberbullying_type
21235,dankmtl yeap as a little propaganda nazi for t...,2
28344,lyndseyboo wow leave lyndsey phone alone bully...,3
31648,im not a man only know how to bully people,3
21779,urgedharry nyazpolitics greenlinerzjm if you a...,2
44850,calling people dumb isnt nice rt tayyoung fuc...,5


## 3.- Pipeline

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

X_train = train_df['tweet_text'].values
y_train = train_df['cyberbullying_type'].values

X_val = val_df['tweet_text'].values
y_val = val_df['cyberbullying_type'].values

X_test = test_df['tweet_text'].values
y_test = test_df['cyberbullying_type'].values

In [18]:
len(X_train), len(X_val), len(X_test)

(33384, 7154, 7154)

In [19]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
raw_val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
raw_test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [20]:
# Define a function to convert the label to a one-hot encoding
def convert_label_to_one_hot(text, label):
    one_hot_label = tf.one_hot(label, 6)
    return text, one_hot_label

# Apply the function to the dataset using map()
raw_train_ds = raw_train_ds.map(convert_label_to_one_hot)
raw_val_ds = raw_val_ds.map(convert_label_to_one_hot)
raw_test_ds = raw_test_ds.map(convert_label_to_one_hot)

batch_size = 32
train_ds = raw_train_ds.shuffle(40538).batch(batch_size)
val_ds = raw_val_ds.batch(batch_size)
test_ds = raw_test_ds.batch(batch_size)

In [21]:
for test_text, test_target in train_ds.take(1):
    print(test_text[0], test_target[0])

tf.Tensor(b'facebook post on giving antiharassment talk at girldevweek how can something be both ironic and apropos httptcoaud0rqnyrz', shape=(), dtype=string) tf.Tensor([0. 0. 0. 1. 0. 0.], shape=(6,), dtype=float32)


## 3.- Modelo

In [22]:
bert_model_path = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_preprocess_path = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [23]:
import tensorflow_hub as hub

### Capa de preprocesamiento

In [24]:
bert_preprocess_model = hub.KerasLayer(bert_preprocess_path)

In [25]:
preprocess_output = bert_preprocess_model(test_text)
preprocess_output

{'input_type_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[  101,  9130,  2695, ...,     0,     0,     0],
        [  101, 22175,  4710, ...,     0,     0,     0],
        [  101,  1045,  2123, ...,     0,     0,     0],
        ...,
        [  101,  5882, 11880, ...,     0,     0,     0],
        [  101,  8840,  2140, ...,     0,     0,     0],
        [  101,  2017,  3726, ...,     0,     0,

### BERT

In [26]:
bert_model = hub.KerasLayer(bert_model_path, trainable=False)
bert_model(preprocess_output)['pooled_output']

<tf.Tensor: shape=(32, 512), dtype=float32, numpy=
array([[ 0.6819405 ,  0.9959505 ,  0.02596496, ...,  0.14001015,
        -0.22612995, -0.69731486],
       [ 0.9780205 ,  0.9879826 ,  0.21857536, ..., -0.09512587,
        -0.25882748, -0.5642965 ],
       [ 0.9914007 ,  0.9833687 ,  0.21825965, ...,  0.36351198,
        -0.21559414, -0.9616104 ],
       ...,
       [ 0.6107497 ,  0.9652837 , -0.11036398, ...,  0.13323538,
        -0.16119605, -0.9608713 ],
       [ 0.98774356,  0.9933103 ,  0.1399471 , ...,  0.3838297 ,
        -0.33955443, -0.65334994],
       [ 0.93482417,  0.9958263 , -0.24297386, ..., -0.03329804,
        -0.25814882, -0.7803331 ]], dtype=float32)>

### Modelo
- Se agrega una capa final a BERT para adaptarlo al dataset de Cyberbullying (6 clases).

In [27]:
def get_model(bert_model_path, bert_preprocess_model, trainable):
    preprocess_model = hub.KerasLayer(bert_preprocess_model)
    bert_model = hub.KerasLayer(bert_model_path, trainable=trainable)
    
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocess_text = bert_preprocess_model(text_input)
    bert_output = bert_model(preprocess_text)['pooled_output']
    x = tf.keras.layers.Dense(128, activation='relu')(bert_output)
    output = tf.keras.layers.Dense(6)(x)
    small_bert = tf.keras.Model(text_input, output)
    return small_bert

freeze_bert = get_model(bert_model_path, bert_preprocess_model, trainable=False)

- Prueba de las salidas de BERT.

In [28]:
freeze_bert(test_text)

<tf.Tensor: shape=(32, 6), dtype=float32, numpy=
array([[-0.02443373,  0.12266673,  0.29725486,  0.6950776 , -0.39368308,
         0.4003989 ],
       [ 0.13776226, -0.2201874 , -0.00256008,  0.64451945, -0.3772892 ,
         0.59791636],
       [ 0.2621168 , -0.21267138,  0.30223024,  0.8603883 , -0.40982962,
         0.60764396],
       [ 0.6784524 ,  0.03175758,  0.15755588,  1.164036  , -0.8846803 ,
         0.67149913],
       [ 0.2489106 , -0.00899002,  0.06998765,  1.344174  , -0.53715855,
         0.76325554],
       [ 0.02273858,  0.13395947, -0.01179492,  0.22953507, -0.18081358,
         0.37057823],
       [ 0.11890835,  0.07106318, -0.08346587,  0.7446954 , -0.07325917,
         0.64346707],
       [-0.5052333 ,  0.5820216 ,  1.1924224 ,  0.50240827, -0.34255046,
        -0.08705637],
       [ 0.15717234,  0.31754464,  0.31166053,  0.6512643 , -0.71804893,
         0.02123843],
       [ 0.7969299 , -0.25651562,  0.366848  ,  1.0768621 , -0.2338041 ,
         0.11402328],
 

- Muestra el tamaño del modelo y los parámetros entrenables.

In [29]:
freeze_bert.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

## 4.- Entrenamiento (última capa)

In [30]:
lr = 0.0001
opt = tf.keras.optimizers.Adam(learning_rate=lr)

freeze_bert.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    optimizer=opt,
                    metrics=['accuracy'])

In [31]:
epochs = 5
history = freeze_bert.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
1044/1044 [==============================] - 67s 63ms/step - loss: 1.0221 - accuracy: 0.6180 - val_loss: 0.7991 - val_accuracy: 0.6953
Epoch 2/5
1044/1044 [==============================] - 58s 56ms/step - loss: 0.7731 - accuracy: 0.7060 - val_loss: 0.7357 - val_accuracy: 0.7179
Epoch 3/5
1044/1044 [==============================] - 60s 57ms/step - loss: 0.7225 - accuracy: 0.7212 - val_loss: 0.6902 - val_accuracy: 0.7339
Epoch 4/5
1044/1044 [==============================] - 61s 59ms/step - loss: 0.6942 - accuracy: 0.7326 - val_loss: 0.6809 - val_accuracy: 0.7414
Epoch 5/5
1044/1044 [==============================] - 65s 62ms/step - loss: 0.6737 - accuracy: 0.7390 - val_loss: 0.6577 - val_accuracy: 0.7477


### Evaluación

In [32]:
freeze_bert.evaluate(test_ds)

224/224 [==============================] - 11s 51ms/step - loss: 0.6700 - accuracy: 0.7434


[0.670005738735199, 0.7433603405952454]

## 6.- Entrenamiento (todas las capas)

In [33]:
full_bert = get_model(bert_model_path, bert_preprocess_model, trainable=True)

In [34]:
full_bert.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [35]:
lr = 0.0001
opt = tf.keras.optimizers.Adam(learning_rate=lr)

full_bert.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    optimizer=opt,
                    metrics=['accuracy'])

In [36]:
epochs = 2
history = full_bert.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/2
1044/1044 [==============================] - 110s 98ms/step - loss: 0.4493 - accuracy: 0.8139 - val_loss: 0.3681 - val_accuracy: 0.8531
Epoch 2/2
1044/1044 [==============================] - 84s 80ms/step - loss: 0.3209 - accuracy: 0.8679 - val_loss: 0.3526 - val_accuracy: 0.8482


### Evaluación

In [37]:
full_bert.evaluate(test_ds)

224/224 [==============================] - 12s 53ms/step - loss: 0.3547 - accuracy: 0.8490


[0.35465729236602783, 0.8490355014801025]

## Ejercicio
- Modifica la arquitectura y el entrenamiento para mejorar los resultados.
- Prueba diferentes versiones de BERT: https://tfhub.dev/google/collections/bert/1.